# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.24.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()


# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'hydraulic-automl-experiment'
project_folder = './pipeline-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
hydraulic-automl-experiment,quick-starts-ws-141045,Link to Azure Machine Learning studio,Link to Documentation


In [3]:
# get the data
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
# Data Source : https://archive.ics.uci.edu/ml/datasets/Condition+monitoring+of+hydraulic+systems
found = False
key = "Hydraulic Systems Data"
description_text = "Condition monitoring of hydraulic systems Data Set"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/chamsun-imoggo/udacityms-3rdproject/main/data/hydraulic_systems_training.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,PS1,PS2,PS3,PS4,PS5,PS6,FS1,FS2,TS1,TS2,TS3,TS4,P1,VS1,CE1,CP1,SE1,STABILITY
count,2094.000000,2094.000000,2094.000000,2094.000000,2094.000000,2094.000000,2094.000000,2094.000000,2094.000000,2094.000000,2094.000000,2094.000000,2094.000000,2094.000000,2094.000000,2094.000000,2094.000000,2094.000000
mean,160.461089,109.383521,1.742380,2.199789,9.122855,9.040759,6.179498,9.621939,45.945370,50.850958,48.145850,41.270196,2492.650573,0.616699,30.479281,1.789732,55.173831,0.360076
std,4.820589,5.116599,0.253804,4.012051,0.563162,0.537043,1.056177,0.444382,7.865514,7.275285,7.336484,7.971456,74.670327,0.059958,11.301936,0.273144,9.175962,0.480137
min,155.391547,104.406307,0.840252,0.000000,8.365801,8.321527,2.018572,8.857513,35.384333,40.978767,38.340283,30.470550,2361.747267,0.524367,17.555983,1.062150,18.276617,0.000000
25%,157.986015,106.913668,1.727014,0.000000,8.535713,8.478621,6.379040,9.197996,36.309758,41.923621,39.204996,31.341463,2441.167950,0.560175,20.032158,1.545979,56.276721,0.000000
50%,158.845991,107.607607,1.766360,0.000000,9.066202,8.985830,6.579164,9.668634,45.556025,50.500100,47.766750,41.061425,2475.461567,0.612275,27.314450,1.734800,58.797425,0.000000
75%,160.983205,109.415065,1.923863,1.266499,9.835300,9.720552,6.655778,10.146995,54.193537,58.723546,55.751542,49.492954,2547.338367,0.653071,46.610821,2.140912,59.687908,1.000000
max,180.922708,131.589089,2.023398,10.182837,9.976781,9.856591,6.722707,10.403098,57.899283,61.958467,59.423167,53.060417,2740.641000,0.839067,47.903667,2.840100,60.755300,1.000000


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "compute-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(ws, amlcompute_cluster_name)
    print(f"{compute_name} exists already")
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_DS14_v2", max_nodes=10)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)
compute_target.wait_for_completion(show_output=True)

Creating....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 60,
    "max_concurrent_iterations": 20,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="STABILITY",
                             blocked_models=['XGBoostClassifier'],
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             enable_onnx_compatible_models=True,
                             **automl_settings
                            )


In [6]:
# TODO: Submit your experiment
automlmodel_run = experiment.submit(automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on compute-ml with default configuration
Running on remote compute: compute-ml
Parent Run ID: AutoML_44b66c17-6335-4af6-b059-70f12198038a

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|3                                |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analy

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
from azureml.widgets import RunDetails
RunDetails(automlmodel_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [8]:
automlmodel_run.wait_for_completion(show_output=True)



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|3                                |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTIO

{'runId': 'AutoML_44b66c17-6335-4af6-b059-70f12198038a',
 'target': 'compute-ml',
 'status': 'Completed',
 'startTimeUtc': '2021-03-21T14:06:26.613037Z',
 'endTimeUtc': '2021-03-21T14:26:38.294302Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'compute-ml',
  'AMLSettingsJsonString': '{"path":null,"name":"hydraulic-automl-experiment","subscription_id":"81cefad3-d2c9-4f77-a466-99a7f541c7bb","resource_group":"aml-quickstarts-141045","workspace_name":"quick-starts-ws-141045","region":"southcentralus","compute_target":"compute-ml","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min"

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
# best run and model for automl
best_run, fitted_model = automlmodel_run.get_output()

# Print the best run
print(best_run)

# Print the fitted_model
print(fitted_model)

# All metrics of the best run
best_run_metrics = best_run.get_metrics()

# Print all metrics of the best run
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Run(Experiment: hydraulic-automl-experiment,
Id: AutoML_44b66c17-6335-4af6-b059-70f12198038a_70,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                objective=None,
                                                                                                random_state=None,
                        

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [10]:
#TODO: Save the best model
model = best_run.register_model(model_name='Hydraulic_BinaryClassification', model_path='outputs/model.pkl')
print(model)

Model(workspace=Workspace.create(name='quick-starts-ws-141045', subscription_id='81cefad3-d2c9-4f77-a466-99a7f541c7bb', resource_group='aml-quickstarts-141045'), name=Hydraulic_BinaryClassification, id=Hydraulic_BinaryClassification:1, version=1, tags={}, properties={})


In [11]:
best_run.download_file('./outputs/model.pkl', output_file_path='./outputs')

In [12]:
from azureml.automl.core.shared import constants
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'automl_env.yml')

In [13]:
# set InferenceConfig
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script='./score.py',
                                    environment=best_run.get_environment())

In [14]:
# deploy the model
from azureml.core import Model
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 2, memory_gb = 1)
service = Model.deploy(workspace=ws, name="automlservice", models=[model], inference_config=inference_config, deployment_config=deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.scoring_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-03-21 14:32:17+00:00 Creating Container Registry if not exists.
2021-03-21 14:32:17+00:00 Registering the environment.
2021-03-21 14:32:19+00:00 Use the existing image.
2021-03-21 14:32:19+00:00 Generating deployment configuration.
2021-03-21 14:32:21+00:00 Submitting deployment to compute..
2021-03-21 14:32:25+00:00 Checking the status of deployment automlservice..
2021-03-21 14:37:28+00:00 Checking the status of inference endpoint automlservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://b4d895f0-87b3-4fa8-8641-cf6145a53c59.southcentralus.azurecontainer.io/score


TODO: In the cell below, send a request to the web service you deployed to test it.

In [21]:
import requests
import json

scoring_uri = service.scoring_uri

# Two sets of data to score, so we get two results back

data={"data":[{"PS1":160.4238,"PS2":109.0127997, "PS3":1.915241667,"PS4":10.12332283,"PS5":9.893727333,"PS6":9.775847667,"FS1":6.400326667,"FS2":10.15790333,"TS1":35.85008333,"TS2":41.44178333,"TS3":38.78375,"TS4":30.89755,"P1":2562.285033,"VS1":0.5509,"CE1":46.97875,"CP1":2.157516667,"SE1":55.78435}]}

# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp)

<Response [200]>


TODO: In the cell below, print the logs of the web service and delete the service

In [20]:
# print service logs
print(service.get_logs())

2021-03-21T14:37:14,838119866+00:00 - iot-server/run 
2021-03-21T14:37:14,839753783+00:00 - rsyslog/run 
2021-03-21T14:37:14,840676493+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-03-21T14:37:14,843487723+00:00 - gunicorn/run 
rsyslogd

<h2>Convert Model to ONNX</h2>

In [22]:
best_run, onnx_model = automlmodel_run.get_output(return_onnx_model=True)

In [23]:
# save the best ONNX model
from azureml.automl.runtime.onnx_convert import OnnxConverter
onnx_file_path = "./automl_onnx_model.onnx"
OnnxConverter.save_onnx_model(onnx_model, onnx_file_path)

In [24]:
# delete the service 
service.delete()

In [25]:
# delete the compute target
compute_target.delete()